In [ ]:
import pyparsing  # make sure you have this installed
import numpy as np
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawHisto import *

# Parse projection string

In [ ]:
finput=ROOT.TFile(" ~/github/data/RootInteractive/testData/JIRA/PWGPP-485/hisPull.root")
hisArray=finput.Get("hisArray")
hisArray.ls()

In [ ]:
np.abs?


In [ ]:
histogramArray={}
for his in hisArray:
    ddHis=thnToNumpyDD(his)
    histogramArray[ddHis['name']]=ddHis

In [ ]:
def buildStr(strToBeBuild):
    if isinstance(strToBeBuild,str):
        return strToBeBuild
    iString=''
    iString += '('
    for sub in strToBeBuild:
        iString += buildStr(sub)
    iString += ')'
    return iString

In [ ]:
def parseExpression(toParse):
    theContent = pyparsing.Word(pyparsing.alphanums+":,;+/-*^.\/")
    parens = pyparsing.nestedExpr("(", ")", content=theContent)
    res = parens.parseString(toParse)
    projection=res.asList()
    projection[0][0]=buildStr(projection[0][0])
    return projection

In [ ]:
expression=parseExpression("( (abs(hisdY)/cos(hisdZ*0.5)) (0:5,0:10,0,0:10) (0) () )")
print(expression)

In [ ]:
def evalHistoExpression(expression, histogramArray):
    # expression  hisdY-hisdZ, abs(hisdY-hisdZ)
    query=expression[0][0]
    keys=list(histogramArray.keys())
    func_list = re.findall(r"\w+\(",expression[0][0])  # there still a paranthesis at the end
    for iKey in keys:
        query=query.replace(iKey,"histogramArray[\'"+iKey+"\'][\'H\']")
    for iFunc in func_list:
        if iFunc[:-1] in dir(np):
            query=query.replace(iFunc,"np."+iFunc)
   # print(query)        
    histogram = eval(query+"[np.index_exp"+str(expression[0][1]).replace("'","")+"]")
    return histogram

In [ ]:
histo=evalHistoExpression(expression, histogramArray)
histo

In [ ]:
toParse0 = "((hisdY/hisdZ)(0:5,0:10,0,0:10)(0)())"
# theContent = pyparsing.Word(pyparsing.alphanums + ".+-_") | '#' | pyparsing.Suppress(',') | pyparsing.Suppress(':')
theContent = pyparsing.Word(pyparsing.alphanums+":,;+/-*^")
parens = pyparsing.nestedExpr("(", ")", content=theContent)
res = parens.parseString(toParse0)
projection=res.asList()
projection

# Parse expression for histograms

In [ ]:
parseExpression("((abs(hisdY))(0:5,0:10,0,0:10)(0)()+abs(hisdZ)(0:5,0:10,0,0:10)(0)()))")

In [ ]:
np.sin((histogramArray['hisdY']['H']))

In [ ]:
(histogramArray['hisdY']['H']*histogramArray['hisdZ']['H'])

# Create slice

In [ ]:
slice = eval("np.index_exp"+str(projection[0][1]).replace("'",""))
slice